In [3]:
# 1. 실수 연산의 함정

a = 0.1
result = 0.0
for i in range(100):
    result +=a

print(result)

9.99999999999998


In [4]:
a = 0.015625
result = 0.0
for i in range(100):
    result +=a

print(result)

1.5625


In [5]:
# 3. 단정도와 배정도

import sys
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

In [6]:
# 배정도 부동수수점으로 표현할 수 있는 가장 큰 수 

sys.float_info.max

1.7976931348623157e+308

In [7]:
# 배정도 부동소수점으로 표현할 수 있는 가장 작은 수 

sys.float_info.min

2.2250738585072014e-308

In [9]:
# 5. 정밀도에 대한 고찰
    
# 5.1 엡실론
    
import sys
sys.float_info.epsilon

2.220446049250313e-16

In [10]:
# 5.2 엡실론과 정밀도

import sys
ep = sys.float_info.epsilon
a = 9.25
diff = (2**3)*ep
diff

1.7763568394002505e-15

In [12]:
b = a+diff
b

9.250000000000002

In [13]:
# 5.2 엡실론과 정밀도 

import sys
ep = sys.float_info.epsilon
diff = (2**3)*ep
diff
a = 9.25
half_diff = diff/2

c = a + half_diff
a == c


True

In [15]:
a = (2.0) ** 53
a


9007199254740992.0

In [17]:
diff_new = (2**53)*ep
diff_new

2.0

In [18]:
b = a+1.0
a == b


True